<a href="https://colab.research.google.com/github/kdemertzis/Earthquakes/blob/main/Euler_Beam_RF_Morf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.integrate import odeint
from sklearn.model_selection import train_test_split, GridSearchCV

# Define the input parameters
E = 29000000.0 # Modulus of elasticity (kN/m2)
I = 0.00423 # Moment of inertia of the cross-section (m4)
Area = 0.69 # Area of the cross-section (m2)
L = 4.0 # Length of beam (m)
p = 27395.515 # Mass/volume unit (kg/m3)
kf = 1500.0 # Modulus of the subgrafe reaction (kN/m2)
Pe = 500.0 # External moving load (kN)
c = 1.00 # Velocity of the external load (m/sec)

# A. STEP: Solution of eigenvalue problem -> Calculation of eigenfrequencies and
# eigenvectors: EI*(d4X/dx4)+(kf-wn2*p*Area)*X=0
# First eigenfrequency
n = 1
wn = np.sqrt((1/(p*Area))*((((n**4)*(np.pi**4)*E*I)/(L**4))+kf))
# First eigenvector - value in the middle of the beam : x=L/2
Pn = ((wn**2*p*Area-kf)/(E*I))**(1/4)
X1L2 = np.sin(Pn*(L/2))

# B. STEP: Calculation of the generalized parameters qi(t)
# (dqi2/dt2)+((wn2*Mn+kf*Nn)/Mn)*qi=(Pe/Mn)*sin(Pn*c*t)
wn2 = wn**2
Nn = (L/2)-(np.sin(2*Pn*L)/(4*Pn))
Mn = p*Area*Nn
k = Pe/Mn
beta = (wn2*Mn+kf*Nn)/Mn
A1mat = np.zeros(801)
A2mat = np.zeros(801)

def damped_vibration(y, t):
    q1 = y[0]
    q2 = y[1]
    dq1dt = q2
    dq2dt = k*np.sin(Pn*c*t) - beta*q1
    return [dq1dt, dq2dt]

initial_conditions = [0, 0]
t = np.linspace(0, 8.0, 801)
q = odeint(damped_vibration, initial_conditions, t)
q1 = q[:, 0]

# Train-test split the data
A1mat = np.arange(0, 8.01, 0.01)
A2mat = X1L2*q1*100
X_train, X_test, y_train, y_test = train_test_split(A1mat.reshape(-1, 1), A2mat.reshape(-1), test_size=0.15, random_state=42)

# Hyperparameter tuning using grid search
rf = RandomForestRegressor(random_state=42)
param_grid = {'n_estimators': [50, 100, 200, 300, 400],
              'max_depth': [5, 10, 15, 20, 25, 30, None],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4]}
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)
rf = grid_search.best_estimator_

# Make predictions on the test set using the trained model
y_pred = rf.predict(X_test)

# Calculate the mean squared error of the predictions on the training set
mse_train = mean_squared_error(y_train, rf.predict(X_train))

# Calculate the mean squared error of the predictions on the test set
mse_test = mean_squared_error(y_test, y_pred)

# Calculate mean absolute error (MAE), root mean squared error (RMSE), and coefficient of determination (R^2)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Print the results
print('Random Forest Best Parameters:')
print(grid_search.best_params_)

print('\nRandom Forest predictions:')
results_df = pd.DataFrame({'Time (sec)': X_test.reshape(-1),
'Displacement (cm) - Actual': y_test,
'Displacement (cm) - Predicted': y_pred})
print(results_df.to_string(index=False))

print('\nRandom Forest Performance Metrics:')
print('Training Set Mean Squared Error:', mse_train)
print('Test Set Mean Squared Error:', mse_test)
print('Mean Absolute Error:', mae)
print('Root Mean Squared Error:', rmse)
print('Coefficient of Determination (R^2):', r2)


Random Forest Best Parameters:
{'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}

Random Forest predictions:
 Time (sec)  Displacement (cm) - Actual  Displacement (cm) - Predicted
       6.97                   -0.171823                      -0.172817
       6.68                   -0.251871                      -0.250701
       0.63                    0.040571                       0.039418
       5.34                   -0.792554                      -0.793422
       0.66                    0.046351                       0.048519
       6.22                   -0.452868                      -0.457791
       3.46                    0.469492                       0.474114
       4.90                   -0.744396                      -0.745841
       5.57                   -0.743117                      -0.742180
       4.56                   -0.565224                      -0.561336
       0.65                    0.044372                       0.042155
   